<h2 id="exercises-i">Exercises I</h2>
<p>Let's review the steps we take at the beginning of each new module.</p>
<ol>
<li>Create a new repository named <code>regression-exercises</code> in your GitHub; all of your Regression work will be housed here.</li>
<li>Clone this repository within your local <code>codeup-data-science</code> directory.</li>
<li>Create a <code>.gitignore</code> and make sure your list of 'files to ignore' includes your <code>env.py</code> file.</li>
<li>Ceate a <code>README.md</code> file that outlines the contents and purpose of your repository.</li>
<li>Add, commit, and push these two files.</li>
<li>Now you can add your <code>env.py</code> file to this repository to access the Codeup database server.</li>
<li>For these exercises, you will create <code>wrangle.ipynb</code> and <code>wrangle.py</code> files to hold necessary functions.</li>
<li>As always, add, commit, and push your work often.</li>
</ol>
<hr />
<h2 id="exercises-ii">Exercises II</h2>
<p>Let's set up an example scenario as perspective for our regression exercises using the Zillow dataset.</p>
<p>As a Codeup data science graduate, you want to show off your skills to the Zillow data science team in hopes of getting an interview for a position you saw pop up on LinkedIn. You thought it might look impressive to build an end-to-end project in which you use some of their Kaggle data to predict property values using some of their available features; who knows, you might even do some feature engineering to blow them away. Your goal is to predict the values of single unit properties using the obervations from 2017.</p>
<p>In these exercises, you will complete the first step toward the above goal: acquire and prepare the necessary Zillow data from the zillow database in the Codeup database server.</p>
<ol>
<li>Acquire <code>bedroomcnt</code>, <code>bathroomcnt</code>, <code>calculatedfinishedsquarefeet</code>, <code>taxvaluedollarcnt</code>, <code>yearbuilt</code>, <code>taxamount</code>, and <code>fips</code> from the <code>zillow</code> database for all 'Single Family Residential' properties.</li>


In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
from env import host, user, password
import zillo_wrangle


In [26]:
# function to contact database
def get_db_url(db_name):
    return f"mysql+pymysql://{user}:{password}@{host}/{db_name}"

In [27]:
# function to query database and return zillow df
def get_zillo():
    query = """
    SELECT bedroomcnt as bedrooms, 
       bathroomcnt as bathrooms,
       calculatedfinishedsquarefeet as square_feet,
       taxvaluedollarcnt as home_value,
       yearbuilt as year,
	   taxamount as taxes,
       fips as fips_number
    FROM predictions_2017
    JOIN properties_2017 USING(id)
    JOIN propertylandusetype USING(propertylandusetypeid)
    WHERE #(transactiondate >= '2017-05-01' AND transactiondate <= '2017-06-30') 
        propertylandusetypeid = '261'
        AND bedroomcnt > 0
        AND bathroomcnt > 0
        AND calculatedfinishedsquarefeet > 0 
        AND taxamount > 0
        AND taxvaluedollarcnt > 0
        AND fips > 0
    ORDER BY fips;
    """
    df = pd.read_sql(query, get_db_url('zillow'))
    return df

In [28]:
zillo = get_zillo()

In [29]:
zillo.head()

,bedrooms,bathrooms,square_feet,home_value,year,taxes,fips_number
0,2.0,2.0,1574.0,728162.0,1937.0,8402.20,6037.0
1,2.0,1.0,1034.0,175643.0,1924.0,2165.10,6037.0
2,4.0,2.0,1722.0,348956.0,1961.0,4238.44,6037.0
3,3.0,2.0,1961.0,812199.0,1948.0,9568.47,6037.0
4,3.0,2.0,1237.0,471166.0,1944.0,5610.38,6037.0


In [30]:
zillo.dtypes

bedrooms       float64
bathrooms      float64
square_feet    float64
home_value     float64
year           float64
taxes          float64
fips_number    float64
dtype: object

In [31]:
# function to clean up my zillow df
def clean_data(df):
    '''
    This funciton takes in the zillow df and drops observations with Null values
    and handles data types returning a df with a basic clean.
    '''
    df = df.dropna()
    df['fips_number'] = df['fips_number'].astype(int)
    df['square_feet'] = df['square_feet'].astype(int)
    df['year'] = df['year'].astype(int)

    return df

In [32]:
zillo = clean_data(zillo)

<ipython-input-31-9325065dbe4a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fips_number'] = df['fips_number'].astype(int)
<ipython-input-31-9325065dbe4a>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['square_feet'] = df['square_feet'].astype(int)
<ipython-input-31-9325065dbe4a>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

In [33]:
zillo.dtypes

bedrooms       float64
bathrooms      float64
square_feet      int64
home_value     float64
year             int64
taxes          float64
fips_number      int64
dtype: object

In [34]:
zillo.describe().T

,count,mean,std,min,25%,50%,75%,max
bedrooms,55588.0,3.311380,0.924005,1.00,3.000,3.000,4.00,25.00
bathrooms,55588.0,2.254884,0.992818,1.00,2.000,2.000,3.00,20.00
square_feet,55588.0,1870.208031,964.140876,320.00,1262.000,1631.000,2217.00,26345.00
home_value,55588.0,456879.285745,593725.070535,9762.00,188731.500,330507.000,534667.50,30166843.00
year,55588.0,1961.232496,21.959743,1862.00,1949.000,1958.000,1976.00,2016.00
taxes,55588.0,5566.992804,6827.036130,20.24,2539.715,4139.215,6415.89,342940.38
fips_number,55588.0,6048.770058,21.129554,6037.00,6037.000,6037.000,6059.00,6111.00


In [35]:
# composite
def wrangle_zillow():
    df = get_data_from_sql()
    df = clean_data(df)
    return df 

In [36]:
zillo.shape

(55588, 7)

In [39]:
zillo.to_csv('zillo_clean.csv', index = False)

2. Using your acquired Zillow data, walk through the summarization and cleaning steps in your <code>wrangle.ipynb</code> file like we did above. You may handle the missing values however you feel is appropriate and meaninful; remember to document your process and decisions using markdown and code commenting where helpful.

3. Store all of the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe witn no missing values in your <code>wrangle.py</code> file. Name your final function <code>wrangle_zillow</code>.